In [3]:
import pandas as pd
import tabula
import OpenDartReader
import requests as rq
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import re
from tqdm import tqdm
import time


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)

def oder_backlog(code,name):

    #기간
    lists = dart.list(code, start='2021-01-01', end ='2024-12-30', kind='A') 
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]
    #수준잔고 확인
    
    rcp_no = []
    for i in range(0,len(lists)):
        rcp = lists['rcept_no'][i]
        rcp_no.append(rcp)  
    print(rcp_no)    

    df = pd.DataFrame()
    comment = ['수주잔고','수주잔액']

    for j in tqdm(range(0,len(rcp_no))):

        files = dart.sub_docs(rcp_no[j])
        
        content = [] #
        content_old = []
        if '매출' in files['title']:
            print('1')
            data = files[files['title'].str.contains('매출')]

            try:
                a = data['url'].values[0]
                res = rq.get(a,headers=headers)
                soup=BeautifulSoup(res.text,'html.parser')
                temp = soup.select('body > table')
                
                for v in temp:
                    text = v.get_text()
                    if ('수주잔고' in text) or ('수주잔액' in text) : 
                        content.append(v)
                
                table = parser.make2d(content[0]) #표로 만들어줌
                adata = pd.DataFrame(table) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
                adata = adata.iloc[-1:,:]
                adata = adata.dropna(axis=1, how="all")
                adata = adata.iloc[:,-1:]
                adata.columns=['수주잔고']
                adata.insert(0,'날짜',lists['공시'][j])
                adata.insert(0,'기업명',name)
                print(adata)
                df = pd.concat([df,adata])
                time.sleep(2)
            
                
            except:
                print('넘어가자')
                    
        else:     
            print('2')  
            data_old = files[files['title'].str.contains('사업의 내용')]

            try:
                b = data_old['url'].values[0]
                res_old = rq.get(b,headers=headers)
                soup_old=BeautifulSoup(res_old.text,'html.parser')
                tem = soup_old.select('body > table')

                for k in tem:
                    text = k.get_text().strip()

                    if ('수주잔고' in text) or ('수주잔액' in text) : 
                        content_old.append(k)
                
                atable = parser.make2d(content_old[0]) #표로 만들어줌
                adata = pd.DataFrame(atable) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
            
                adata = adata.iloc[-1:,:]
                adata = adata.dropna(axis=1, how="all")
                adata = adata.iloc[:,-1:]
                adata.columns=['수주잔고']
                adata.insert(0,'날짜',lists['공시'][j])
                adata.insert(0,'기업명',name)
                df = pd.concat([df,adata])
                time.sleep(2)
            except:
                print('이것도 넘어가자')
    # df.to_clipboard()
    return df

def save(corpData,name):
    file_name = './수주잔고/수주잔고.xlsx'
    writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
    corpData.to_excel(
        writer, 
        sheet_name=name,
        startcol = 0,
        startrow = writer.sheets[name].max_row,
        index=False, 
        na_rep = '',      # 결측값을 ''으로 채우기
        inf_rep = '',     # 무한값을 ''으로 채우기
        header = None
        )
    writer.close()
    
if __name__ == '__main__':
    corp = {
        # '131290':'티에스이',
        # '042660':'한화오션',
        '103590':'일진전기',
    }
    
    for code,name in corp.items():
        print(name)
        corpData = oder_backlog(code,name)
        save(corpData,name)
        

일진전기
['20241113000067', '20240813000314', '20240514000243', '20240313001767', '20231113000277', '20230811000829', '20230512000636', '20230315000560', '20221114000951', '20220816000581', '20220516000285', '20220317001272', '20211112000551', '20210813000420', '20210517000023', '20210329000663']


  0%|          | 0/16 [00:00<?, ?it/s]

2


  6%|▋         | 1/16 [00:02<00:33,  2.23s/it]

2


...

2


 94%|█████████▍| 15/16 [00:33<00:02,  2.27s/it]

2


100%|██████████| 16/16 [00:36<00:00,  2.26s/it]


KeyError: '일진전기'

In [4]:
corpData.to_clipboard()

In [9]:
lists = dart.list('267260', start='2018-05-01', end ='2023-09-01', kind='A')  #'044490'
lists['공시'] = lists['report_nm'].str.split(' ').str[1]
files = dart.sub_docs('20230811001535')
data = files[files['title'].str.contains('수주상황')]

a = data['url'].values[0]
res = rq.get(a)
soup=BeautifulSoup(res.text,'html.parser')
temp = soup.select('body > table')
for v in temp:
    text = v.get_text()
    if comment in text : 
        content.append(v)
table = parser.make2d(content[0]) #표로 만들어줌
data = pd.DataFrame(table) # 필요한부분만 불러옴
data.columns = data.iloc[0]
data = data.iloc[-1:,:]
data['날짜'] = lists['공시'][j]
df = pd.concat([df,data])
                

NameError: name 'comment' is not defined

In [ ]:
#'20230811000549', '20230512000749', '20230316000298'

lists = dart.list('073010', start='2018-05-01', end ='2023-09-01', kind='A')  #'044490'
lists['공시'] = lists['report_nm'].str.split(' ').str[1]
files = dart.sub_docs('20230811000549')

if '수주상황' in files['title'][12]:
    print('수주상황')
else:
    print('없음')    

# data = files[files['title'].str.contains('수주상황')]
# a = data['url'].values[0]
# res = rq.get(a)
# soup=BeautifulSoup(res.text,'html.parser')
# temp = soup.select('body > table')
# comment = '수주잔고'
# content = [] #
# for v in temp:
#     text = v.get_text()
#     if comment in text : 
#         content.append(v)
# table = parser.make2d(content[0])
# adata = pd.DataFrame(table)
# adata.columns = adata.iloc[0]
# adata=adata.iloc[-1:,:]

#'20230811000549', '20230512000749', '20230316000298'
# adata


수주상황


In [ ]:
def oder_backlog(corp_name):

    #기간
    lists = dart.list(corp_name, start='2020-05-01', end ='2023-12-15', kind='A') 
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]

    #수준잔고 확인
    rcp_no = []
    for i in range(0,len(lists)):
        rcp = lists['rcept_no'][i]
        rcp_no.append(rcp)  
        

    df = pd.DataFrame()
    comment = '수주잔고'

    for j in tqdm(range(0,len(rcp_no))):
             
        files = dart.sub_docs(rcp_no[j])
        content = [] #
        content_old = []
        if '제품' in files['title']:

            data = files[files['title'].str.contains('제품')]
            try:
                a = data['url'].values[0]
                res = rq.get(a,headers=headers)
                soup=BeautifulSoup(res.text,'html.parser')
                temp = soup.select('body > table')
                
                for v in temp:
                    text = v.get_text()
                    if comment in text : 
                        content.append(v)
                table = parser.make2d(content[0]) #표로 만들어줌
                adata = pd.DataFrame(table) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
                adata = adata.iloc[-1:,:]
                adata['날짜'] = lists['공시'][j]
                adata['기업명'] = corp_name
                
                df = pd.concat([df,adata])
                time.sleep(2)
                
            except:
                print('넘어가자')
                    
        else:     
                
            data_old = files[files['title'].str.contains('사업의 내용')]
            try:
                b = data_old['url'].values[0]
                res_old = rq.get(b,headers=headers)
                soup_old=BeautifulSoup(res_old.text,'html.parser')
                tem = soup_old.select('body > table')
                for k in tem:
                    text = k.get_text()
                    if comment in text : 
                        content_old.append(k)

                atable = parser.make2d(content_old[0]) #표로 만들어줌
                adata = pd.DataFrame(atable) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
                adata = adata.iloc[-1:,:]
                adata['날짜'] = lists['공시'][j]
                adata['기업명'] = corp_name
                
                df = pd.concat([df,adata])
                time.sleep(2)
            except:
                print('이것도 넘어가자')
    df.to_clipboard()
    return df

corp_name = oder_backlog('쎄트렉아이')
corp_name.to_clipboard()

100%|██████████| 15/15 [00:34<00:00,  2.32s/it]


0   품목 수주일자   납기 수주총액   수주총액 기납품액   기납품액 수주잔고   수주잔고         날짜    기업명
3  합 계  합 계  합 계    -  6,686    -  4,705    -  1,981  (2023.09)  쎄트렉아이
3  합 계  합 계  합 계    -  6,081    -  3,876    -  2,205  (2023.06)  쎄트렉아이
3  합 계  합 계  합 계    -  5,950    -  3,454    -  2,496  (2023.03)  쎄트렉아이
3  합 계  합 계  합 계    -  5,278    -  2,720    -  2,558  (2022.12)  쎄트렉아이
3  합 계  합 계  합 계    -  4,862    -  3,272    -  1,590  (2022.09)  쎄트렉아이
3  합 계  합 계  합 계    -  4,500    -  2,828    -  1,673  (2022.06)  쎄트렉아이
3  합 계  합 계  합 계    -  4,358    -  2,746    -  1,612  (2022.03)  쎄트렉아이
3  합 계  합 계  합 계    -  4,422    -  2,720    -  1,702  (2021.12)  쎄트렉아이
3  합 계  합 계  합 계    -  4,236    -  2,536    -  1,700  (2021.09)  쎄트렉아이
3  합 계  합 계  합 계    -  4,186    -  2,342    -  1,844  (2021.06)  쎄트렉아이
3  합 계  합 계  합 계    -  3,982    -  2,117    -  1,865  (2021.03)  쎄트렉아이
3  합 계  합 계  합 계    -  4,225    -  2,206    -  2,019  (2020.12)  쎄트렉아이
3  합 계  합 계  합 계    -  3,992    -  1,917    -  2,075  (2020.09)  쎄트렉아이
3  합 계  합 계  합 계    -  3,145    -  1,717    -  1,428  (2020.06)  쎄트렉아이
3  합 계  합 계  합 계    -  3,065    -  1,504    -  1,561  (2020.03)  쎄트렉아이

In [3]:
corp_name.to_clipboard()

In [7]:

def oder_backlog1(corp_name):

    #기간
    lists = dart.list(corp_name, start='2018-05-01', end ='2023-09-01', kind='A') 
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]

    #수준잔고 확인
    rcp_no = []
    for i in range(0,len(lists)):
        rcp = lists['rcept_no'][i]
        rcp_no.append(rcp)  
        

    df = pd.DataFrame()
    comment = '수주잔고'

    for j in tqdm(range(0,len(rcp_no))):
            
        files = dart.sub_docs(rcp_no[j])
        content = [] #
        content_old = []
        if '수주' in files['title']:

            data = files[files['title'].str.contains('수주상황')]
            try:
                a = data['url'].values[0]
                res = rq.get(a,headers=headers)
                soup=BeautifulSoup(res.text,'html.parser')
                temp = soup.select('body > table')
                
                for v in temp:
                    text = v.get_text()
                    if comment in text : 
                        content.append(v)
                table = parser.make2d(content[0]) #표로 만들어줌
                adata = pd.DataFrame(table) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
                adata = adata.iloc[-1:,:]
                adata['날짜'] = lists['공시'][j]
                adata['기업명'] = corp_name
                
                df = pd.concat([df,adata])
                time.sleep(2)
                
            except:
                print('넘어가자')
                    
        else:     
                
            data_old = files[files['title'].str.contains('사업의 내용')]
            try:
                b = data_old['url'].values[0]
                res_old = rq.get(b,headers=headers)
                soup_old=BeautifulSoup(res_old.text,'html.parser')
                tem = soup_old.select('body > table')
                for k in tem:
                    text = k.get_text()
                    if comment in text : 
                        content_old.append(k)

                atable = parser.make2d(content_old[0]) #표로 만들어줌
                adata = pd.DataFrame(atable) # 필요한부분만 불러옴
                adata.columns = adata.iloc[0]
                adata = adata.iloc[-1:,:]
                adata['날짜'] = lists['공시'][j]
                adata['기업명'] = corp_name
                
                df = pd.concat([df,adata])
                time.sleep(2)
            except:
                print('이것도 넘어가자')
    # df.to_clipboard()
    return df

# corp_name = oder_backlog1('이수페타시스')
# corp_name.to_clipboard()
# corp_name = corp_name[['기업명','날짜','수주잔고']]
# corp_name = corp_name.sort_values(by='날짜',ascending=True)

In [10]:
corp_name.to_clipboard()

In [8]:

corp_list = pd.read_excel('종목리스트.xlsx',sheet_name=0)
corp_suju=pd.DataFrame()

for i in tqdm(corp_list['Name']):
    print(i)
    try:
        corp_name = oder_backlog1(i)
        corp_name = corp_name[['기업명','날짜','수주잔고']]
        corp_name = corp_name.sort_values(by='날짜',ascending=True)
        cor = corp_name.T
        cor.columns = cor.iloc[1]
        cor = cor.drop(['기업명','날짜'],axis=0)
        cor = cor.rename(index={'수주잔고':i})
        corp_suju = pd.concat([corp_suju,cor])
    except:
        print('ff')
corp_suju.to_clipboard()


  0%|          | 0/3 [00:00<?, ?it/s]

HD현대중공업


이것도 넘어가자


 33%|███▎      | 1/3 [00:23<00:46, 23.38s/it]

이것도 넘어가자
HJ중공업


 67%|██████▋   | 2/3 [01:20<00:43, 43.47s/it]

HSD엔진


100%|██████████| 3/3 [02:14<00:00, 44.89s/it]


In [8]:
corp_suju.to_clipboard()

In [6]:
stack = corp_name[corp_name.iloc[:,0].str.contains('합 계')]
# stack = stack[['수주잔고','(2023.06)']]
stack = stack.iloc[:,-2:]
stack['(2023.06)'] = stack['(2023.06)'].str.replace('(',"")
stack['(2023.06)'] = stack['(2023.06)'].str.replace(')',"")
stack.set_index('(2023.06)',inplace=True)
stack = stack.apply(pd.to_numeric,errors='ignore')
stack

NameError: name 'corp_name' is not defined

In [ ]:

#수준잔고 확인
rcp_no = []
for i in range(0,len(lists)):
    rcp = lists['rcept_no'][i]
    rcp_no.append(rcp)

df1 = pd.DataFrame()

for k in tqdm(range(0,len(rcp_no))):
    files = dart.sub_docs(rcp_no[k])
    data = files[files['title'].str.contains('사업의 내용')]
    a = data['url'].values[0]
    res = rq.get(a)
    soup=BeautifulSoup(res.text,'html.parser')
    tem = soup.select('body > table')
    b = []
    for v in tem:
        text = v.get_text()
        if '수주잔고' in text : 
            b.append(v)

    atable = parser.make2d(b[0]) #표로 만들어줌
    adata = pd.DataFrame(atable).set_index(0) # 필요한부분만 불러옴
    adata['날짜'] = lists['공시'][k]
    df1 = pd.concat([df1,adata])

df1.to_clipboard()


  0%|          | 0/14 [00:00<?, ?it/s]


KeyError: '공시'

In [43]:
#매출확인

def sales(stock_name):
    lists = dart.list(stock_name, start='2023-05-01', end ='2023-09-01', kind='A')  #'044490'
    lists['공시'] = lists['report_nm'].str.split(' ').str[1]
    # 사전 데이터 확인
    files = dart.sub_docs(lists['rcept_no'][0])
    data = files[files['title'].str.contains('수주상황')]
    a = data['url'].values[0]
    res = rq.get(a)
    soup=BeautifulSoup(res.text,'html.parser')
    temp = soup.find_all('table')
    table = parser.make2d(temp[1]) #표로 만들어줌
    data = pd.DataFrame(table).set_index(0) # 필요한부분만 불러옴
    data['날짜'] = lists['공시'][0]
    data['기업명'] = stock_name
    return 

ValueError: Cannot set a DataFrame with multiple columns to the single column 기업명